Das brauche ich um's nicht zu vergessen.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns


In [18]:
df = pd.read_csv('/Users/whypk/01Projekte/air_quality/data/waqi-covid19-airqualitydata-2025.csv', comment='#')
df.sample(20)

,Date,Country,City,Specie,count,min,max,median,variance
581657,2024-10-31,ES,Pamplona,no2,48,6.4,28.4,15.6,257.10
1320745,2024-05-05,CO,Medellín,no2,48,2.3,23.8,10.6,418.00
2085902,2024-01-19,JP,Yokohama,pm10,536,1.0,40.0,13.0,448.92
421925,2024-01-17,FR,Nantes,pm25,48,10.0,51.0,34.0,932.23
377582,2024-10-29,DE,Freiburg,humidity,72,53.7,94.7,86.0,1539.60
2285988,2025-02-12,GB,Swansea,no2,96,3.3,14.5,8.9,110.81
1306377,2024-04-25,CL,Concepción,pressure,168,1015.0,1017.0,1016.0,3.35
1806690,2024-01-05,KZ,Nur-Sultan,dew,23,-8.5,-2.0,-5.5,47.09
1395163,2024-08-28,FI,Tampere,wind-speed,96,2.3,6.6,3.3,23.74
269145,2024-10-26,US,Tucson,pressure,144,1015.8,1023.3,1019.4,25.89


In [20]:
mask_top_five_cities = (df['City'] == "Abu Dhabi") | (df['City'] == "Buenos Aires") | (df['City'] == "Vienna") | (df['City'] == "Brisbane") | (df['City'] == "Sarajevo")
df_top_five_cities = df[mask_top_five_cities]
df_top_five_cities.sample(50)

,Date,Country,City,Specie,count,min,max,median,variance
1952225,2024-03-28,AE,Abu Dhabi,so2,189,1.8,16.0,7.2,150.05
999623,2024-04-24,AT,Vienna,no2,288,1.9,19.2,6.0,93.39
1471896,2024-03-14,AR,Buenos Aires,precipitation,17,0.1,12.9,1.5,111.98
1363354,2024-11-03,BA,Sarajevo,humidity,144,54.0,100.0,100.0,3671.64
1367391,2024-07-12,BA,Sarajevo,so2,73,0.6,24.4,2.6,96.07
319216,2024-02-09,AU,Brisbane,wd,207,92.0,234.0,185.0,8665.26
1471970,2024-07-10,AR,Buenos Aires,pm10,36,31.0,49.0,42.0,267.49
1469264,2024-09-03,AR,Buenos Aires,wind-gust,6,0.2,0.7,0.6,0.56
1949857,2025-01-31,AE,Abu Dhabi,temperature,192,18.0,22.0,20.0,17.40
1367399,2025-02-17,BA,Sarajevo,so2,102,6.2,59.8,20.3,1833.89


In [31]:
mask_vienna = (df['City'] == "Vienna") & (df['Date'] == "2024-01-01")& (df['Country'] == "AT")
df_vienna = df[mask_vienna]
print(df_vienna)

               Date Country    City         Specie  count    min     max  \
996426   2024-01-01      AT  Vienna           pm10    356    1.0    49.0   
996763   2024-01-01      AT  Vienna       humidity    408   57.3    95.8   
997373   2024-01-01      AT  Vienna            so2     59    0.6    51.7   
997467   2024-01-01      AT  Vienna       pressure    408  983.9  1021.2   
997863   2024-01-01      AT  Vienna  precipitation     36    0.1    99.6   
998235   2024-01-01      AT  Vienna     wind-speed    402    0.2    22.2   
998702   2024-01-01      AT  Vienna             co     36    0.1     0.1   
999348   2024-01-01      AT  Vienna    temperature    408    2.8     9.4   
999713   2024-01-01      AT  Vienna            no2    351    1.0    19.9   
1000017  2024-01-01      AT  Vienna      wind-gust    336    0.8    43.5   
1000420  2024-01-01      AT  Vienna            dew    249   -0.5     3.4   
1000735  2024-01-01      AT  Vienna           pm25    296    5.0   107.0   

         me

In [22]:
mask_london = (df['City'] == "London") & (df['Date'] == "2025-01-08")& (df['Country'] == "GB")
df_london = df[mask_london]
print(df_london)

               Date Country    City       Specie  count    min     max  \
2240923  2025-01-08      GB  London   wind-speed    473    0.1     5.7   
2241376  2025-01-08      GB  London    wind-gust    349    0.2    10.8   
2241510  2025-01-08      GB  London           co    192    1.5     6.8   
2241860  2025-01-08      GB  London  temperature    576   -2.0     3.0   
2242542  2025-01-08      GB  London         pm10    483    2.0    52.0   
2242637  2025-01-08      GB  London     humidity    576   80.0    99.0   
2243236  2025-01-08      GB  London          no2    462    4.3    38.0   
2243853  2025-01-08      GB  London          dew     72   -1.0     1.0   
2244234  2025-01-08      GB  London           o3    226    0.6    20.8   
2244584  2025-01-08      GB  London          so2    167    0.3     2.7   
2244883  2025-01-08      GB  London         pm25    479    5.0    97.0   
2245376  2025-01-08      GB  London     pressure    576  997.2  1030.7   

         median  variance  
2240923  

In [25]:
#df_london = df_london.drop(columns=['variance', 'min', 'max'])
df_london

,Date,Country,City,Specie,count,median
2240923,2025-01-08,GB,London,wind-speed,473,2.2
2241376,2025-01-08,GB,London,wind-gust,349,5.3
2241510,2025-01-08,GB,London,co,192,4.6
2241860,2025-01-08,GB,London,temperature,576,1.1
2242542,2025-01-08,GB,London,pm10,483,14.0
2242637,2025-01-08,GB,London,humidity,576,90.0
2243236,2025-01-08,GB,London,no2,462,21.5
2243853,2025-01-08,GB,London,dew,72,0.0
2244234,2025-01-08,GB,London,o3,226,6.0
2244584,2025-01-08,GB,London,so2,167,1.0


In [29]:
df_pivot_london = df_london.pivot(index=["Date", "Country", "City"], columns="Specie", values=["median", 'count']).reset_index()
df_pivot_london.columns = ['_'.join(col).strip('_') for col in df_pivot_london.columns]
df_pivot_london

,Date,Country,City,median_co,median_dew,median_humidity,median_no2,median_o3,median_pm10,median_pm25,...,count_humidity,count_no2,count_o3,count_pm10,count_pm25,count_pressure,count_so2,count_temperature,count_wind-gust,count_wind-speed
0,2025-01-08,GB,London,4.6,0.0,90.0,21.5,6.0,14.0,39.0,...,576.0,462.0,226.0,483.0,479.0,576.0,167.0,576.0,349.0,473.0


In [30]:
df_top_five_cities_pivot = df_top_five_cities.pivot(index=["Date", "Country", "City"], columns="Specie", values=["median", 'count']).reset_index()
df_top_five_cities_pivot.columns = ['_'.join(col).strip('_') for col in df_top_five_cities_pivot.columns]
df_top_five_cities_pivot


,Date,Country,City,median_co,median_dew,median_humidity,median_no2,median_o3,median_pm10,median_pm25,...,count_o3,count_pm10,count_pm25,count_precipitation,count_pressure,count_so2,count_temperature,count_wd,count_wind-gust,count_wind-speed
0,2024-01-01,AE,Abu Dhabi,0.1,19.0,73.0,8.1,21.2,50.0,53.0,...,119.0,120.0,24.0,NaN,192.0,120.0,192.0,NaN,NaN,184.0
1,2024-01-01,AR,Buenos Aires,1.9,15.8,63.0,4.6,12.2,14.0,34.0,...,24.0,75.0,24.0,2.0,92.0,8.0,92.0,21.0,48.0,92.0
2,2024-01-01,AT,Vienna,0.1,1.9,72.7,4.2,NaN,6.0,17.0,...,NaN,356.0,296.0,36.0,408.0,59.0,408.0,NaN,336.0,402.0
3,2024-01-01,AU,Brisbane,1.5,20.4,87.0,1.9,10.6,9.0,15.0,...,72.0,215.0,212.0,102.0,216.0,5.0,208.0,216.0,26.0,216.0
4,2024-01-01,BA,Sarajevo,1.6,6.0,96.5,15.6,25.2,21.0,76.0,...,144.0,121.0,98.0,6.0,144.0,115.0,144.0,NaN,6.0,144.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2025-02-17,AR,Buenos Aires,NaN,19.4,87.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,26.0,NaN,26.0,NaN,NaN,13.0
2053,2025-02-17,AT,Vienna,0.1,-9.0,70.2,9.6,NaN,26.0,76.0,...,NaN,235.0,216.0,NaN,265.0,43.0,265.0,NaN,152.0,157.0
2054,2025-02-17,AU,Brisbane,0.7,16.4,58.5,2.3,8.6,23.0,27.0,...,69.0,216.0,216.0,NaN,203.0,19.0,214.0,192.0,24.0,214.0
2055,2025-02-17,BA,Sarajevo,3.7,-4.0,86.0,12.4,15.6,31.0,91.0,...,86.0,102.0,101.0,NaN,107.0,102.0,107.0,NaN,NaN,107.0
